In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
%pwd

'd:\\ML_Projects\\Violence-Reporter'

In [19]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir:Path
    base_model_path:Path
    updated_base_model_path: Path
    params_image_height: int
    params_image_width: int
    params_sequence_length: int
    params_include_top: bool
    params_weights: str
    params_istrainable: bool
    params_freeze_till: int
    params_classes: int

In [20]:
from violenceReporter.constants import *
from violenceReporter.utils.common import read_yaml,create_directories

In [21]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    

    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        
        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_include_top=self.params.INCLUDE_TOP,
            params_weights=self.params.WEIGHTS,
            params_classes=self.params.CLASSES,
            params_image_height= self.params.IMAGE_HEIGHT,
            params_image_width= self.params.IMAGE_WIDTH,
            params_sequence_length= self.params.SEQUENCE_LENGTH,
            params_istrainable= self.params.TRAINABLE,
            params_freeze_till= self.params.FREEZE_TILL
        )

        return prepare_base_model_config

In [22]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import *
from keras.models import Sequential
from keras.applications.mobilenet_v2 import MobileNetV2

In [26]:
class PrepareBaseModel:
    def __init__(self,config:PrepareBaseModelConfig):
        self.config=config
    
    def get_base_model(self):
        self.model = MobileNetV2(
            include_top=self.config.params_include_top, 
            weights=self.config.params_weights)
        
        self.model.trainable = self.config.params_istrainable

        self.save_model(self.config.base_model_path,model=self.model)
    
    @staticmethod
    def prepare_full_model(base_model,classes,freeze_till,sequence_length,image_height,image_width):
        for layer in base_model.layers[:freeze_till]:
            layer.trainable=False
        

        model = Sequential()
        model.add(Input(shape=(sequence_length,image_height,image_width, 3)))
        model.add(TimeDistributed(base_model))
        model.add(Dropout(0.25))
        model.add(TimeDistributed(Flatten()))

        lstm_fw = LSTM(units=32)
        lstm_bw = LSTM(units=32, go_backwards=True)
        model.add(Bidirectional(lstm_fw, backward_layer=lstm_bw))

        model.add(Dropout(0.25))
        model.add(Dense(256, activation='relu'))
        model.add(Dropout(0.25))
        model.add(Dense(128, activation='relu'))
        model.add(Dropout(0.25))
        model.add(Dense(64, activation='relu'))
        model.add(Dropout(0.25))
        model.add(Dense(32, activation='relu'))
        model.add(Dropout(0.25))
        model.add(Dense(classes, activation='softmax'))
        

        model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=["accuracy"])

        model.summary()

        return model
    
    def update_base_model(self):
        self.full_model = self.prepare_full_model(
            base_model=self.model,
            classes=self.config.params_classes,
            freeze_till=self.config.params_freeze_till,
            sequence_length=self.config.params_sequence_length,
            image_height=self.config.params_image_height,
            image_width=self.config.params_image_width
        )

        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

In [27]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e

[2024-03-31 01:28:03,393: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-03-31 01:28:03,400: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-31 01:28:03,402: INFO: common: created directory at: artifacts]
[2024-03-31 01:28:03,405: INFO: common: created directory at: artifacts/prepare_base_model]
[2024-03-31 01:28:03,408: WARNING: mobilenet_v2: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.]


[2024-03-31 01:28:05,488: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed (TimeDistr  (None, 16, 2, 2, 1280)   2257984   
 ibuted)                                                         
                                                                 
 dropout (Dropout)           (None, 16, 2, 2, 1280)    0         
                                                                 
 time_distributed_1 (TimeDis  (None, 16, 5120)         0         
 tributed)                                                       
                                                                 
 bidirectional (Bidirectiona  (None, 64)               1319168   
 l)                                                  